In [ ]:
%pip install semver

In [ ]:
import semver
import shutil
import os
import json


In [ ]:
def read_value_from_json(file_path, key):
    with open(file_path, 'r') as file:
        data = json.load(file)
        value = data[key]
        return value
    
def read_values_from_json(file_path, keys):
    with open(file_path, 'r') as file:
        data = json.load(file)
        values = []
        for key in keys:
            values.append(data[key])
        return values
    
# ver = read_value_from_json('src/info.json', 'version')
# appname = read_value_from_json('src/info.json', 'name')
# print(ver, appname)

In [ ]:
def write_kv_to_json(file_path, key, value):
    with open(file_path, 'r') as file:
        data = json.load(file)
        data[key] = value
        with open(file_path, 'w') as file:
            json.dump(data, file, indent=4)

def write_kvs_to_json(file_path, keys, values):
    with open(file_path, 'r') as file:
        data = json.load(file)
        for i in range(len(keys)):
            data[keys[i]] = values[i]
        with open(file_path, 'w') as file:
            json.dump(data, file, indent=4)

# write_kv_to_json('src/info.json', 'version', '0.0.5')
# write_kv_to_json('src/info.json', 'version', '3.14.159')
            

In [ ]:
def zip_app():
    appname, version = read_values_from_json('mod/src/info.json', ['name', 'version'])
    zip_filename = f'{appname}_{version}'
    zip_path = f'{zip_filename}'
    shutil.make_archive(zip_path, 'zip', 'mod')
    return zip_path, appname


In [ ]:
def bump_version(file_path, type='patch'):
    current_version = read_value_from_json(file_path, 'version')
    if type == 'major':
        new_version = semver.bump_major(current_version)
    elif type == 'minor':
        new_version = semver.bump_minor(current_version)
    elif type == 'patch':
        new_version = semver.bump_patch(current_version)
    else:
        raise ValueError('Invalid type')
    write_kv_to_json(file_path, 'version', new_version)
    return new_version

# bump_version('src/info.json')


In [ ]:
def find_other_installed_versions(mod_folder):
    files = os.listdir(mod_folder)
    appname, version = read_values_from_json('mod/src/info.json', ['name', 'version'])
    other_versions = [f for f in files if appname in f and version not in f]
    return other_versions

# find_other_installed_versions(factorio_mods)

In [ ]:
def remove_old_versions(mod_folder):
    other_versions = find_other_installed_versions(mod_folder)
    for version in other_versions:
        os.remove(mod_folder + version)
    return other_versions

# remove_old_versions(factorio_mods)

In [ ]:
def package_app(mod_folder):
    version = bump_version('mod/src/info.json')
    path, appname = zip_app()
    dest = shutil.copy(f'{path}.zip', mod_folder)
    old_versions = remove_old_versions(mod_folder)
    return f'Packaged {appname} {version} to {dest} and removed {old_versions}'


In [ ]:
factorio_mods = 'C:/Users/peter/AppData/Roaming/Factorio/mods/'
package_app(factorio_mods)

In [ ]:
%pip install pyautogui

In [ ]:
import pyautogui


In [ ]:

def find_window(window_title):
    window = pyautogui.getWindowsWithTitle(window_title)
    if window:
        return window[0]
    else:
        return None

def focus_window(window_title):
    window = find_window(window_title)
    if window:
        window.activate()
        return True
    else:
        return False
    


In [ ]:
win_name = 'Factorio 1.1.101'


In [ ]:
factorio_window = find_window(win_name)

In [ ]:
def click_text_in_window(text):
    try:
        text_location = pyautogui.locateCenterOnScreen(f'ref/{text}.png')
        if text_location:
            pyautogui.click(text_location)
            return True
        else:
            return False
    except pyautogui.ImageNotFoundException:
        return False


In [ ]:
def click_exit_from_main_menu():
    try:
        # assumes a png file named 'exit.png' exists in the 'ref' folder w a screenshot of factorio's exit button
        click_text_in_window('exit')
        return True
    except pyautogui.ImageNotFoundException:
        return False
    
# click_exit_from_main_menu()

In [ ]:
from datetime import datetime

In [ ]:
def get_changelog():
    # check if change log exists
    # if not, create it
    # if it does, read it and return the contents
    changelog_path = 'mod/src/changelog.txt'
    if not os.path.exists(changelog_path):
        with open(changelog_path, 'w') as file:
            file.write('')
        return ''
    else:
        with open(changelog_path, 'r') as file:
            return file.read()
        

def write_changelog(changelog):
    changelog_path = 'mod/src/changelog.txt'
    with open(changelog_path, 'w') as file:
        file.write(changelog)

def write_to_changelog(text):
    changelog = get_changelog()
    changelog += text
    write_changelog(changelog)

def format_output(categories):
    """
    Takes a dictionary with categories and lists of entries and formats them into a structured string.
    
    Args:
    - categories (dict): A dictionary where keys are category names and values are lists of entries.
    
    Returns:
    - str: A formatted string with categories and entries.
    """
    output = []
    for category, entries in categories.items():
        output.append(f"{category}:")
        for entry in entries:
            # For multi-line entries, ensure proper indentation for lines after the first
            formatted_entry = entry.replace("\n", "\n    ").rstrip()
            output.append(f"      - {formatted_entry}")
    return "\n".join(output)

def create_changelog_entry(params):
    changelog = []
    changelog.append('-'*99 + '\n')
    version_string = f'Version: {read_value_from_json("mod/src/info.json", "version")}\n'
    date_string = f"Date: {datetime.now():%d. %m. %Y}\n"
    changelog.append(version_string)
    changelog.append(date_string)
    for changes in format_output(params):
        changelog.append(changes)
    changelog.append('\n')
    return ''.join(changelog)

def add_changelog_entry(params):
    entry = create_changelog_entry(params)
    write_to_changelog(entry)
    return entry

# add_changelog_entry({'Added': ['New feature', 'Another new feature'], 'Changed': ['Updated feature', 'Another updated feature'], 'Fixed': ['Bug fix', 'Another bug fix']})

In [ ]:
import re
from collections import Counter

def extract_repeated_timestamps(filenames):
    """
    Extracts and returns a list of unique timestamps that are repeated in the filenames,
    assuming the timestamp format is an integer sequence before the '-' character.
    
    Args:
    - filenames (list of str): The list of filenames in the format 'timestamp-suffix.ext'.
    
    Returns:
    - list of int: A list of unique, repeated timestamps.
    """
    # Regular expression to match the timestamp pattern
    timestamp_pattern = re.compile(r'^(\d+)-')
    
    timestamps = []
    for filename in filenames:
        match = timestamp_pattern.match(filename)
        if match:
            timestamps.append(int(match.group(1)))
    
    # Count occurrences of each timestamp
    timestamp_counts = Counter(timestamps)
    
    # Filter timestamps that are repeated, i.e., count > 1
    repeated_timestamps = [timestamp for timestamp, count in timestamp_counts.items() if count > 1]
    
    return repeated_timestamps

In [ ]:
script_output_folder = 'C:/Users/peter/AppData/Roaming/Factorio/script-output'
extract_repeated_timestamps(os.listdir(script_output_folder))